In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install thop

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from thop import profile
import os

In [ ]:
def print_shapes(x, stage):
    print(f"Stage: {stage} | Shape: {x.shape} | Channels: {x.shape[1]}")

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(channels, channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(channels),
            nn.ReLU(inplace=True)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(channels, channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(channels)
        )
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.conv2(out)
        out += identity
        return self.relu(out)

class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.preconv = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.res1 = ResidualBlock(out_channels)
        self.res2 = ResidualBlock(out_channels)

    def forward(self, x):
        identity = self.preconv(x)
        out = self.res1(identity)
        out = self.res2(out)
        return out + identity

class DoubleConvUp(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.block(x)


class Down(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.AvgPool2d(kernel_size=2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)

class Up(nn.Module):
    def __init__(self, up_channels, skip_channels, out_channels, bilinear=True):
        super().__init__()

        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(up_channels, up_channels, kernel_size=2, stride=2)

        self.reduce_channels = nn.Conv2d(up_channels + skip_channels, out_channels, kernel_size=1)
        self.conv = DoubleConv(out_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]
        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        x = torch.cat([x1, x2], dim=1)
        x = self.reduce_channels(x)
        return self.conv(x)

class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

In [ ]:
class UNet(nn.Module):
    def __init__(self, n_channels, bilinear=False):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 16)
        self.down1 = (Down(16, 32))
        self.down2 = (Down(32, 64))
        self.down3 = (Down(64, 128))
        factor = 2 if bilinear else 1
        self.up1 = Up(128, 64, 64, bilinear)
        self.up2 = Up(64, 32, 32, bilinear)
        self.up3 = Up(32, 16, 16, bilinear)
        self.outc = (OutConv(16, 1))

    def forward(self, x):
        x1 = model.inc(x)
        x2 = model.down1(x1)
        x3 = model.down2(x2)
        x4 = model.down3(x3)
        x = model.up1(x4, x3)
        x = model.up2(x, x2)
        x = model.up3(x, x1)
        x = model.outc(x)
        return x

    def use_checkpointing(self):
        self.inc = torch.utils.checkpoint(self.inc)
        self.down1 = torch.utils.checkpoint(self.down1)
        self.down2 = torch.utils.checkpoint(self.down2)
        self.down3 = torch.utils.checkpoint(self.down3)
        self.down4 = torch.utils.checkpoint(self.down4)
        self.up1 = torch.utils.checkpoint(self.up1)
        self.up2 = torch.utils.checkpoint(self.up2)
        self.up3 = torch.utils.checkpoint(self.up3)
        self.up4 = torch.utils.checkpoint(self.up4)
        self.outc = torch.utils.checkpoint(self.outc)

Read the Data

In [2]:
from osgeo import gdal

def read_modis_var_from_hdf_gdal(filepath, var_name):
    dataset = gdal.Open(filepath)
    if dataset is None:
        raise ValueError(f"Impossible d'ouvrir {filepath}")

    subdatasets = dataset.GetSubDatasets()
    print("\n".join(f"{name} | {desc}" for name, desc in subdatasets))
    for name, desc in subdatasets:
        if var_name in name or var_name in desc:
            ds = gdal.Open(name)
            array = ds.ReadAsArray()
            return array
    raise ValueError(f"Variable {var_name} non trouvée dans {filepath}")

In [6]:
read_modis_var_from_hdf_gdal("/content/drive/MyDrive/MLA/Project/data/hdf_files/MOD09GQ.A2017002.h18v04.061.2021363134842.hdf","QC_250m_1")

ValueError: Impossible d'ouvrir /content/drive/MyDrive/MLA/Project/data/hdf_files/MOD09GQ.A2017002.h18v04.061.2021363134842.hdf

Converting them into Tensors

In [ ]:
import numpy as np

def extract_valid_windows(array, window_size=32, target_value=4096):
    no_cloud_windows = []
    h, w = array.shape

    for i in range(0, h - window_size + 1, window_size):
        for j in range(0, w - window_size + 1, window_size):
            window = array[i:i+window_size, j:j+window_size]

            if np.all(window == target_value):
                no_cloud_windows.append(window)

    return no_cloud_windows

array = np.full((512, 512), 4096, dtype=np.uint16)

no_cloud_windows = extract_valid_windows(array, window_size=32, target_value=4096)

print(f"Nombre de fenêtres valides : {len(no_cloud_windows)}")


Nombre de fenêtres valides : 256


**Dataset TIFF definition**

In [ ]:
import pandas as pd
from torch.utils.data import Dataset
import torchvision.transforms as T
import tifffile

In [ ]:
def normalize(img):
    mean = img.mean()
    std = img.std()
    return (img - mean) / std

In [ ]:
class LSTNDVIDataset(Dataset):
    def __init__(self, csv_file, transform=None, base_dir="/content/drive/MyDrive/MLA/Project"):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.base_dir = base_dir

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        lst_path = os.path.join(self.base_dir, self.data.iloc[idx, 1])
        ndvi_path = os.path.join(self.base_dir, self.data.iloc[idx, 2])

        lst = tifffile.imread(lst_path)
        ndvi = tifffile.imread(ndvi_path)

        lst = torch.tensor(lst, dtype=torch.float32).unsqueeze(0)
        ndvi = torch.tensor(ndvi, dtype=torch.float32).unsqueeze(0)

        lst = normalize(lst)
        ndvi = normalize(ndvi)

        return ndvi, lst

**Running the U-Net**

In [ ]:
def degrade_image(img, scale=0.5):
    h, w = img.shape[2:]
    new_h, new_w = int(h * scale), int(w * scale)
    lowres = F.interpolate(img, size=(new_h, new_w), mode='bilinear', align_corners=False)
    upscaled = F.interpolate(lowres, size=(h, w), mode='bilinear', align_corners=False)
    return upscaled

In [ ]:
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

model = UNet(n_channels=2, bilinear=True)
model = model.to("cpu")

ndvi = torch.randn(1, 4, 256, 256)
lst = torch.randn(1, 4, 256, 256)
input = torch.cat([ndvi, lst], dim=1)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataset = LSTNDVIDataset("/content/drive/MyDrive/MLA/Project/data/pairs_day.csv")
loader = DataLoader(dataset, batch_size=4, shuffle=True)

n_epochs = 1

In [ ]:
def sobel_gradient(x):
    # x : [B, C, H, W]
    sobel_x = torch.tensor([[[-1, 0, 1],
                             [-2, 0, 2],
                             [-1, 0, 1]]], dtype=torch.float32, device=x.device).unsqueeze(0)
    sobel_y = torch.tensor([[[-1, -2, -1],
                             [ 0,  0,  0],
                             [ 1,  2,  1]]], dtype=torch.float32, device=x.device).unsqueeze(0)

    # Appliquer sur chaque canal
    gradients_x = F.conv2d(x, sobel_x.expand(x.shape[1], 1, 3, 3), padding=1, groups=x.shape[1])
    gradients_y = F.conv2d(x, sobel_y.expand(x.shape[1], 1, 3, 3), padding=1, groups=x.shape[1])

    return gradients_x, gradients_y

In [ ]:
for epoch in range(n_epochs):
    model.train()
    # i = 0
    for ndvi, lst in loader:
        # if i == 100:
        #     break
        # i += 1
        target_size = ndvi.shape[2:]

        if ndvi.dim() == 3:
            ndvi = ndvi.unsqueeze(0)
        if lst.dim() == 3:
            lst = lst.unsqueeze(0)

        lst = F.interpolate(
            lst,
            size=ndvi.shape[2:],
            mode='bilinear',
            align_corners=False
        )

        ndvi, lst = ndvi.to(device), lst.to(device)
        input = torch.cat([ndvi, lst], dim=1)
        output = model(input)

        ndvi_grad_x, ndvi_grad_y = sobel_gradient(ndvi)
        output_grad_x, output_grad_y = sobel_gradient(output)

        ndvi_grad = torch.sqrt(ndvi_grad_x**2 + ndvi_grad_y**2)
        output_grad = torch.sqrt(output_grad_x**2 + output_grad_y**2)

        loss = criterion(output, lst)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        scheduler.step()


    print(f"Epoch {epoch+1}: Loss = {loss.item():.4f}")

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [ ]:
import matplotlib.pyplot as plt
from torchvision.transforms.functional import to_pil_image


model.eval()

n_examples = 5
displayed = 0


with torch.no_grad():
    for data in loader:
        ndvi, lst = data[0].to("cpu"),  data[1].to("cpu")

        if ndvi.dim() == 3:
            ndvi = ndvi.unsqueeze(0)
        if lst.dim() == 3:
            lst = lst.unsqueeze(0)

        lst = F.interpolate(lst, size=ndvi.shape[2:], mode='bilinear', align_corners=False)
        input = torch.cat([ndvi, lst], dim=1)
        pred = model(input)

        loss = F.mse_loss(pred, lst)
        print("MSE sur la sortie du modèle vs LST réelle :", loss.item())

        batch_size = ndvi.shape[0]
        for b in range(batch_size):
            if displayed >= n_examples:
                break

            fig, axs = plt.subplots(1, 3, figsize=(12, 4))
            axs[0].imshow(ndvi[b, 0], cmap='viridis')
            axs[0].set_title("NDVI")
            axs[1].imshow(lst[b, 0], cmap='inferno')
            axs[1].set_title("LST Réelle")
            axs[2].imshow(pred[b, 0], cmap='inferno')
            axs[2].set_title("LST Prédite")

            for ax in axs:
                ax.axis("off")
            plt.tight_layout()
            plt.show()

            displayed += 1

        if displayed >= n_examples:
            break


for data in loader:
    ndvi, lst = data[0].to(device),  data[1].to(device)

    if ndvi.dim() == 3:
        ndvi = ndvi.unsqueeze(0)
    if lst.dim() == 3:
        lst = lst.unsqueeze(0)

    lst = F.interpolate(lst, size=ndvi.shape[2:], mode='bilinear', align_corners=False)
    input = torch.cat([ndvi, lst], dim=1)
    macs, params = profile(model, inputs=(input[0].unsqueeze(0),))
    break

print(f"Nombre de MACs : {macs:,}")
print(f"Nombre de paramètres : {params:,}")


In [ ]:
i = 0

for ndvi, lst in loader:
    mean = 0.
    var = 0.
    nb_pixels = 0

    if i == 100:
        break
    batch = ndvi
    batch = batch.to(torch.float32)

    n = batch.numel()

    mean += batch.sum().item()
    var += (batch ** 2).sum().item()
    nb_pixels += n
    i += 1

    mean /= nb_pixels
    var = var / nb_pixels - mean ** 2
    std = var ** 0.5

    print(f"Mean: {mean:.4f}")
    print(f"Std : {std:.4f}")
